In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from geopy.geocoders import Nominatim
import pandas as pd
import time

In [2]:
opciones=Options()

opciones.add_experimental_option('excludeSwitches', ['enable-automation'])
opciones.add_experimental_option('useAutomationExtension', False)

opciones.headless=False    # si True, no aperece la ventana (headless=no visible)

opciones.add_argument('--start-maximized')         # comienza maximizado

opciones.add_argument('user-data-dir=selenium')    # mantiene las cookies

opciones.add_argument('--incognito')              # incognito


PATH='./chromedriver'
driver=webdriver.Chrome(PATH, options=opciones)
driver.get('https://www.spain.info/es/resultados-busqueda/index.html?q=sevilla&reloaded&page=1&rpp=36&subTypeFilter=Museo,Monumento,Jard%C3%ADn%20hist%C3%B3rico')

In [3]:
texto_columnas = driver.find_elements_by_class_name('title')

In [6]:
frase = []
for i in texto_columnas:
    frase.append(i.text)

frase = frase[11:17]
frase

['Setas de Sevilla',
 'Puerta de Sevilla',
 'Ayuntamiento de Sevilla',
 'Catedral de Sevilla',
 'Real Alcázar de Sevilla',
 'Museo Arqueológico de Sevilla']

In [7]:
direccion = []
for i in range(0,len(frase)):
    geolocator = Nominatim(user_agent="edu")
    location = geolocator.geocode(frase[i])
    direccion.append(location.address)
    time.sleep(3)

In [8]:
direccion

['Setas de Sevilla, Plaza de la Encarnación, Encarnación-Regina, Casco Antiguo, Sevilla, Andalucía, 41003, España',
 'Puerta de Sevilla, Calle Puerta Sevilla, San Basilio, Santiago, Distrito Centro, Córdoba, Andalucía, España',
 'Ayuntamiento de Sevilla, 1, Plaza Nueva, El Arenal, Casco Antiguo, Sevilla, Andalucía, 41001, España',
 'Catedral de Sevilla, Plaza del Triunfo, Santa Cruz, Casco Antiguo, Sevilla, Andalucía, España',
 'Real Alcázar de Sevilla, Patio de Banderas, Santa Cruz, Casco Antiguo, Sevilla, Andalucía, 41004, España',
 'Museo Arqueológico, Plaza de América, El Prado-Parque de María Luisa, Distrito Sur, Sevilla, Andalucía, 41005, España']

In [18]:
direccione = []
for i in direccion:
    corte = i.split(",")[1:]
    direccione.append(corte)

direccione

[[' Plaza de la Encarnación',
  ' Encarnación-Regina',
  ' Casco Antiguo',
  ' Sevilla',
  ' Andalucía',
  ' 41003',
  ' España'],
 [' Calle Puerta Sevilla',
  ' San Basilio',
  ' Santiago',
  ' Distrito Centro',
  ' Córdoba',
  ' Andalucía',
  ' España'],
 [' 1',
  ' Plaza Nueva',
  ' El Arenal',
  ' Casco Antiguo',
  ' Sevilla',
  ' Andalucía',
  ' 41001',
  ' España'],
 [' Plaza del Triunfo',
  ' Santa Cruz',
  ' Casco Antiguo',
  ' Sevilla',
  ' Andalucía',
  ' España'],
 [' Patio de Banderas',
  ' Santa Cruz',
  ' Casco Antiguo',
  ' Sevilla',
  ' Andalucía',
  ' 41004',
  ' España'],
 [' Plaza de América',
  ' El Prado-Parque de María Luisa',
  ' Distrito Sur',
  ' Sevilla',
  ' Andalucía',
  ' 41005',
  ' España']]

In [19]:
df = pd.DataFrame(list(zip(frase,direccione)), columns = ['Visitar','Direccion'])
df

,Visitar,Direccion
0,Setas de Sevilla,"[ Plaza de la Encarnación, Encarnación-Regina..."
1,Puerta de Sevilla,"[ Calle Puerta Sevilla, San Basilio, Santiag..."
2,Ayuntamiento de Sevilla,"[ 1, Plaza Nueva, El Arenal, Casco Antiguo,..."
3,Catedral de Sevilla,"[ Plaza del Triunfo, Santa Cruz, Casco Antig..."
4,Real Alcázar de Sevilla,"[ Patio de Banderas, Santa Cruz, Casco Antig..."
5,Museo Arqueológico de Sevilla,"[ Plaza de América, El Prado-Parque de María ..."
